In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

# Drawing

In [2]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
# boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['boiling'] = boiling_plan_df['sku'].apply(lambda sku: sku.boilings[0])
boiling_plan_df.sort_values(by='id')

def add_push_with_props(parent, block):
    return add_push(parent, block, new_props=block.props.get_all_props())

mode = 'prod'
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

root = make_schedule(boiling_plan_df)

In [3]:
with make('schedule', orient='vertical'):
    make(orient='vertical', h=1, visible=False)
    with make('header', orient='vertical', h=1, index_width=3):
        make(h=1, size=1, text='График наливов', push_func=add_push)
        for i in range(288):
            cur_time = cast_time(i + cast_t('01:00'))
            if cur_time[-2:] == '00':
                make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    for i in range(2):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)

    with make('cleanings', orient='vertical', h=2):
        for block in root['cleaning']:
            make(block, push_func=add_push_with_props)
    make(orient='vertical', h=1, visible=False)

    for i in range(2, 4):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)
    
    with make('header', orient='vertical', h=1, index_width=4):
        for i in range(288):
            cur_time = cast_time(i + cast_t('07:00'))
            if cur_time[-2:] == '00':
                make(t=i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    with make('water_melting', orient='vertical', h=4):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['melting'], push_func=add_push_with_props)
                
    with make('water_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
    
    # todo: make dynamic lines
    n_lines = 5
    melting_lines = [make(f'salt_melting_{i}', orient='vertical', h=3).block for i in range(n_lines)]
    
    # todo: hardcode, add empty elements for drawing not to draw melting_line itself
    for b in melting_lines:
        add_push(b, Block('block', visible=False))
        
    for i, block in enumerate([b for b in root['boiling'] if b.props['boiling_type'] == 'salt']):
        add_push_with_props(melting_lines[i % n_lines], block['melting_and_packing']['melting'])
    
    with make('salt_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'salt':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
row_root = maker.root

In [4]:
row_root.props.update({'h': max(c.end for c in root.children)})
style = load_style()
init_sheet_func = init_sheet if mode == mode else init_template_sheet(template_fn=template_fn)
wb = draw_schedule(row_root, style, init_sheet_func=init_sheet_func)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 

In [ ]:
row_root